In [1]:
import sys
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import ast

sys.path.append(os.path.abspath('../src/'))
from ActivationFunctions import *
from NeuralNetwork import *
from MyUtils import *
from ModelSelection import *

%matplotlib inline

In [2]:
tr_norm_df = pd.read_csv('../data/divided_std_train_0_8.csv')
test_norm_df = pd.read_csv('../data/divided_std_test_0_2.csv')

tr_df = pd.read_csv('../data/divided_train_0_8.csv')
test_df = pd.read_csv('../data/divided_test_0_2.csv')

TR_INPUT = 10
TR_OUTPUT = 3

tr_normalized = tr_norm_df.values
test_normalized = test_norm_df.values
training_set = tr_df.values
test_set = test_df.values

scaler_out = StandardScaler()
scaler_out.fit(tr_normalized[:,TR_INPUT:])

training_len = len(tr_norm_df)
test_len = len(test_norm_df)

In [4]:
def remove_useless_val(x):
    a =['learning_rate', 'lr_decay_tau', 'alpha_momentum']
    b =['adamax_learning_rate', 'exp_decay_rate_1', 'exp_decay_rate_2']
    if x['adamax']:
        for i in a:
            if i in x.keys():
                x[i] = None
    else:
        for i in b:
            if i in x.keys():
                x[i] = None   
    return x

In [5]:
columns_order = ['topology', 'stats',
 'batch_size',
 'min_epochs',
 'max_epochs',
 'patience',
 'error_increase_tolerance',
 'lambda_tikhonov',
 
 'learning_rate',
 'alpha_momentum',
 'lr_decay_tau',
 
 'adamax',
 'adamax_learning_rate',
 'exp_decay_rate_1',
 'exp_decay_rate_2',
 
 'mean_mean_euclidean_error',
 'mean_mean_squared_error',
 'var_mean_euclidean_error',
 'var_mean_squared_error',
 'mean_best_validation_training_error']

In [14]:
results_files = ['multy_topology_sigmoid_4.csv']
topologies = ['wrong']
folder = '../data/gs_data/'
topologies_dict = {}
gs_results = []
names = ['20_sigmoid', '6_6_6_sigmoid', '12_12_sigmoid', '12_sigmoid', '40_sigmoid']
for i, f in enumerate(results_files):
    if os.path.isfile(folder+ f):
        dummy = pd.read_csv(folder + f)
        topologies_dict[topologies[i]] = ast.literal_eval(dummy['topology'][0])
        dummy['topology'] = ['20_sigmoid', '6_6_6_sigmoid', '12_12_sigmoid', '12_sigmoid', '40_sigmoid']

        gs_results.append(dummy)
    

orig_df = pd.concat(gs_results)
orig_df

,adamax,alpha_momentum,batch_size,error_increase_tolerance,lambda_tikhonov,learning_rate,lr_decay_tau,max_epochs,min_epochs,patience,topology,stats,mean_mean_euclidean_error,mean_mean_squared_error,var_mean_euclidean_error,var_mean_squared_error,mean_best_validation_training_error
0,False,0.9,10,0.00001,0.000001,0.1,175,600,100,5,20_sigmoid,"{'error_increase_tolerance': 1e-05, 'exp_decay...",0.121779,0.019921,0.000109,1.121264e-05,0.017031
1,False,0.9,10,0.00001,0.000001,0.1,175,600,100,5,6_6_6_sigmoid,"{'error_increase_tolerance': 1e-05, 'exp_decay...",1.069022,1.421318,0.001646,1.684536e-02,1.371320
2,False,0.9,10,0.00001,0.000001,0.1,175,600,100,5,12_12_sigmoid,"{'error_increase_tolerance': 1e-05, 'exp_decay...",0.461650,0.264049,0.003136,3.735190e-03,0.245064
3,False,0.9,10,0.00001,0.000001,0.1,175,600,100,5,12_sigmoid,"{'error_increase_tolerance': 1e-05, 'exp_decay...",0.151632,0.030697,0.000015,5.139484e-07,0.025322
4,False,0.9,10,0.00001,0.000001,0.1,175,600,100,5,40_sigmoid,"{'minibatch_size': 10, 'exp_decay_rate_2': 0.9...",0.114844,0.018009,0.000007,1.969571e-06,0.013112


In [15]:
columns_order = [x for x in columns_order if x in orig_df.columns]
orig_df = orig_df[columns_order]

order_by = 'mean_mean_euclidean_error'
orig_df.sort_values(by=[order_by], inplace=True)
orig_df = orig_df.reset_index(drop=True)
gs_results = orig_df.drop(['stats'], axis=1)

In [16]:
gs_results = gs_results.apply(remove_useless_val, axis=1)
for i in gs_results.columns[1:]:
    gs_results[i] = gs_results[i].astype(float)

In [17]:
numerical_col = list(gs_results.columns)
numerical_col.remove('topology')
st_opt_col = ['learning_rate','lr_decay_tau','alpha_momentum']
adamax_opt_col = ['adamax','adamax_learning_rate','exp_decay_rate_1','exp_decay_rate_2']
metrics_col = [x for x in gs_results.columns if x.startswith(('var', 'mean'))]
general_col = [item for item in list(gs_results.columns) if item not in st_opt_col and item not in metrics_col and item not in adamax_opt_col]
tr_input_col = [item for item in list(gs_results.columns) if item not in metrics_col]

In [18]:
error_treshold = 10
gs_results_clean = gs_results.loc[(gs_results['mean_mean_euclidean_error'].notna()) & (gs_results['mean_mean_euclidean_error'] < error_treshold)].copy()


gs_results_clean = gs_results_clean.reset_index(drop=True)
gs_results_clean.style.background_gradient(cmap='Blues')

,topology,batch_size,min_epochs,max_epochs,patience,error_increase_tolerance,lambda_tikhonov,learning_rate,alpha_momentum,lr_decay_tau,adamax,mean_mean_euclidean_error,mean_mean_squared_error,var_mean_euclidean_error,var_mean_squared_error,mean_best_validation_training_error
0,40_sigmoid,10.000000,100.000000,600.000000,5.000000,0.000010,0.000001,0.100000,0.900000,175.000000,0.000000,0.114844,0.018009,0.000007,0.000002,0.013112
1,20_sigmoid,10.000000,100.000000,600.000000,5.000000,0.000010,0.000001,0.100000,0.900000,175.000000,0.000000,0.121779,0.019921,0.000109,0.000011,0.017031
2,12_sigmoid,10.000000,100.000000,600.000000,5.000000,0.000010,0.000001,0.100000,0.900000,175.000000,0.000000,0.151632,0.030697,0.000015,0.000001,0.025322
3,12_12_sigmoid,10.000000,100.000000,600.000000,5.000000,0.000010,0.000001,0.100000,0.900000,175.000000,0.000000,0.461650,0.264049,0.003136,0.003735,0.245064
4,6_6_6_sigmoid,10.000000,100.000000,600.000000,5.000000,0.000010,0.000001,0.100000,0.900000,175.000000,0.000000,1.069022,1.421318,0.001646,0.016845,1.371320
